In [1]:
from datasets import load_dataset

squad = load_dataset("squad", split="train")
squad = squad.train_test_split(test_size=0.2, seed=42)

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [2]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')  # optional, sometimes needed for tokenization

[nltk_data] Downloading package wordnet to c:\Users\Jettanat\anaconda3
[nltk_data]     \envs\fuckingneuralnetwork\lib\nltk_data...
[nltk_data] Downloading package omw-1.4 to c:\Users\Jettanat\anaconda3
[nltk_data]     \envs\fuckingneuralnetwork\lib\nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to c:\Users
[nltk_data]     \Jettanat\anaconda3\envs\fuckingneuralnetwork\lib\nltk
[nltk_data]     _data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to c:\Users
[nltk_data]     \Jettanat\anaconda3\envs\fuckingneuralnetwork\lib\nltk
[nltk_data]     _data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to c:\Users\Jettanat\anaconda3\e
[nltk_data]     nvs\fuckingneuralnetwork\lib\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [5]:
from nltk import pos_tag

words = ['Slavic ']
tags = pos_tag(words)
print(tags)

[('Slavic ', 'NN')]


SYNONYM REPLACEMENT

In [6]:
import random
from random import shuffle
from nltk.corpus import wordnet 
def get_only_chars(line):

    clean_line = ""

    line = line.replace("’", "")
    line = line.replace("'", "")
    line = line.replace("-", " ") #replace hyphens with spaces
    line = line.replace("\t", " ")
    line = line.replace("\n", " ")
    line = line.lower()

    for char in line:
        if char in 'qwertyuiopasdfghjklzxcvbnm ':
            clean_line += char
        else:
            clean_line += ' '

    clean_line = re.sub(' +',' ',clean_line) #delete extra spaces
    if clean_line[0] == ' ':
        clean_line = clean_line[1:]
    return clean_line

stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 
			'ours', 'ourselves', 'you', 'your', 'yours', 
			'yourself', 'yourselves', 'he', 'him', 'his', 
			'himself', 'she', 'her', 'hers', 'herself', 
			'it', 'its', 'itself', 'they', 'them', 'their', 
			'theirs', 'themselves', 'what', 'which', 'who', 
			'whom', 'this', 'that', 'these', 'those', 'am', 
			'is', 'are', 'was', 'were', 'be', 'been', 'being', 
			'have', 'has', 'had', 'having', 'do', 'does', 'did',
			'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or',
			'because', 'as', 'until', 'while', 'of', 'at', 
			'by', 'for', 'with', 'about', 'against', 'between',
			'into', 'through', 'during', 'before', 'after', 
			'above', 'below', 'to', 'from', 'up', 'down', 'in',
			'out', 'on', 'off', 'over', 'under', 'again', 
			'further', 'then', 'once', 'here', 'there', 'when', 
			'where', 'why', 'how', 'all', 'any', 'both', 'each', 
			'few', 'more', 'most', 'other', 'some', 'such', 'no', 
			'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 
			'very', 's', 't', 'can', 'will', 'just', 'don', 
			'should', 'now', '']
def synonym_replacement(words, n):
    new_words = words.copy()
    
    # POS tagging
    tags = pos_tag(words)
    
    # Candidate words: skip stopwords, proper nouns (NNP/NNPS), numbers
    candidates = [
        word for word, pos in tags
        if word.lower() not in stop_words
        and pos not in ["NNP", "NNPS"]   # skip proper nouns
        and not word.isdigit()           # skip numbers
    ]
    
    random.shuffle(candidates)
    num_replaced = 0
    
    for word in candidates:
        synonyms = get_synonyms(word)
        if synonyms:
            synonym = random.choice(synonyms)
            new_words = [synonym if w == word else w for w in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break

    # Reconstruct sentence
    sentence = ' '.join(new_words)
    new_words = sentence.split(' ')

    return new_words

def get_synonyms(word):
	synonyms = set()
	for syn in wordnet.synsets(word): 
		for l in syn.lemmas(): 
			synonym = l.name().replace("_", " ").replace("-", " ").lower()
			synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
			synonyms.add(synonym) 
	if word in synonyms:
		synonyms.remove(word)
	return list(synonyms)

In [7]:
import random
import re
from nltk.corpus import wordnet
from copy import deepcopy

# squad = load_dataset("squad")
examples_dict = squad['train'][:]

# Convert column-oriented dict to list of row dicts
examples = [dict(zip(examples_dict.keys(), values)) for values in zip(*examples_dict.values())]

# Augmentation parameters
N_AUGMENTATIONS_PER_SAMPLE = 1
MAX_REPLACEMENTS = 1
random.seed(1)

augmented_samples = []

for ex in examples:
    original_q = ex['question']
    cleaned = get_only_chars(original_q)
    words = cleaned.split(' ')

    # Create augmented versions
    for i in range(N_AUGMENTATIONS_PER_SAMPLE):
        augmented_words = synonym_replacement(words, MAX_REPLACEMENTS)
        augmented_q = ' '.join(augmented_words)

        new_sample = deepcopy(ex)
        new_sample['id'] = ex['id'] + f"_aug_{i}"
        new_sample['question'] = augmented_q
        augmented_samples.append(new_sample)

# Print results
print(f"Original examples: {len(examples)}")
print(f"Augmented examples: {len(augmented_samples)}\n")

for i, ex in enumerate(examples[1:5]):
    print(f"🟩 Original:  {ex['question']}")
    for aug in augmented_samples:
        if aug['id'].startswith(ex['id']):
            print(f"🟦 Augmented: {aug['question']}")
    print('-' * 80)


Original examples: 70079
Augmented examples: 70079

🟩 Original:  What was the price for early HDTVs?
🟦 Augmented: what was the toll for early hdtvs 
--------------------------------------------------------------------------------
🟩 Original:  At its zenith, the imperialist forces controlled all but how many states in Mexico?
🟦 Augmented: at its zenith the imperialist strength controlled all but how many states in mexico 
--------------------------------------------------------------------------------
🟩 Original:  Serbo-Croatian is the only Slavic language to use what two scripts together?
🟦 Augmented: serbo croatian is the only slavic language to use what two script together 
--------------------------------------------------------------------------------
🟩 Original:  What is the third hottest desert in the world?
🟦 Augmented: what is the third blistering desert in the world 
--------------------------------------------------------------------------------


In [8]:
from datasets import Dataset, DatasetDict
import pandas as pd

# Convert augmented samples (list of dicts) to a Dataset
augmented_train_dataset = Dataset.from_pandas(pd.DataFrame(augmented_samples))

# Keep original test dataset
test_dataset = squad['test']

# Create new DatasetDict
augmented_squad = DatasetDict({
    'train': augmented_train_dataset,
    'test': test_dataset
})

# Check
print(squad['train']['question'][4])
print(augmented_squad['train']['question'][4])

What is the third hottest desert in the world?
what is the third blistering desert in the world 


In [9]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [10]:
tokenized_squad = augmented_squad.map(preprocess_function, batched=True, remove_columns=augmented_squad["train"].column_names)

Map:   0%|          | 0/70079 [00:00<?, ? examples/s]

In [11]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("./drill04_qa_model")

# Freeze first 2 transformer layers
for layer in model.distilbert.transformer.layer[:2]:
    for param in layer.parameters():
        param.requires_grad = False

# Check which parameters are trainable
trainable_params = [n for n, p in model.named_parameters() if p.requires_grad]
print(f"Trainable parameters: {len(trainable_params)}")


Trainable parameters: 70


In [12]:
# ✅ Use GPU if available
import torch
from transformers import Trainer, TrainingArguments

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"There are {torch.cuda.device_count()} GPU(s) available.")
    print("Device name:", torch.cuda.get_device_name(0))
    model.to(device)
else:
    device = torch.device("cpu")
    print("No GPU found, using CPU.")

# ✅ Training configuration (optimized for disk usage)
training_args = TrainingArguments(
    output_dir="drill04+QuestionSynonym",       # folder to save model
    eval_strategy="epoch",         # correct parameter name
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    save_strategy="epoch",               # save checkpoint only once per epoch
    save_total_limit=1,                  # keep only the last checkpoint
    load_best_model_at_end=True,         # optional, keeps best checkpoint
    push_to_hub=False,
    report_to="none",                    # disable wandb or tensorboard logs
    logging_dir=None,                    # avoid creating logging folders
)

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,                 # fixed from "processing_class"
    data_collator=data_collator,
)

# ✅ Try to resume training if a checkpoint exists
import os
last_checkpoint = None
if os.path.isdir(training_args.output_dir):
    from transformers.trainer_utils import get_last_checkpoint
    last_checkpoint = get_last_checkpoint(training_args.output_dir)

if last_checkpoint:
    print(f"Resuming training from checkpoint: {last_checkpoint}")
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    print("Starting training from scratch...")
    trainer.train()


There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3050 Laptop GPU


C:\Users\Jettanat\AppData\Local\Temp\ipykernel_33024\1267413822.py:32: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Starting training from scratch...


Epoch,Training Loss,Validation Loss
1,1.139700,1.125495
2,0.990200,1.126850
3,0.912400,1.152648
4,0.820300,1.169969


In [13]:
# Save the model and tokenizer
trainer.save_model("./drill04+QuestionSynonym")
tokenizer.save_pretrained("./drill04+QuestionSynonym")

('./drill04+QuestionSynonym\\tokenizer_config.json',
 './drill04+QuestionSynonym\\special_tokens_map.json',
 './drill04+QuestionSynonym\\vocab.txt',
 './drill04+QuestionSynonym\\added_tokens.json',
 './drill04+QuestionSynonym\\tokenizer.json')

In [14]:
max_length = 384
stride = 128


def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

validation_dataset = squad["test"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=squad["test"].column_names,
)
len(squad["test"]), len(validation_dataset)

Map:   0%|          | 0/17520 [00:00<?, ? examples/s]

(17520, 17688)

In [15]:
from tqdm.auto import tqdm
import collections
import numpy as np
import evaluate
metric = evaluate.load("squad")

n_best = 20
max_answer_length = 30

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [16]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, squad["test"])

  0%|          | 0/17520 [00:00<?, ?it/s]

{'exact_match': 63.06506849315068, 'f1': 76.98337549190808}

In [36]:
question = 'which car costs $20000'
context="The blue car costs $20,000 and the red car costs $25,000."
print(question)
from transformers import pipeline
question_answerer = pipeline("question-answering",
                             model="./drill04+QuestionSynonym",
                            tokenizer="./drill04+QuestionSynonym",
                            fp16=True)

result = question_answerer(question=question, context=context)
print(result['answer'])

Device set to use cuda:0


which car costs $20000
red car


In [39]:
augmented_train_dataset['question'][1:100]

['what was the toll for early hdtvs ',
 'at its zenith the imperialist strength controlled all but how many states in mexico ',
 'serbo croatian is the only slavic language to use what two script together ',
 'what is the third blistering desert in the world ',
 'what is the nearest port to angel helena ',
 'what did darwin introduce that changed classification system ',
 'how many legislative assemblies are held each yr ',
 'when was thatcher appointed prime parson ',
 'what type of hold do communitarians tend to support ',
 'in what year did sir john herschel publish his work on infrared radiation ',
 'who sack the petition of the jewish community to stop the abuse of them ',
 'what nasser related film set a box agency record ',
 'in foot how much of the dawlish sea wall was destroyed ',
 'how very much was the average cost of hospital stays for asthma related issues for children ',
 'for what state artifacts did the vestal care ',
 'with what belorussian metropolis does kathmandu ha